<a href="https://colab.research.google.com/github/Super-rookie-Py/_WebProgramming_Class/blob/master/_%EB%B0%95%EA%B1%B4%EC%9A%B0_%5B%EA%B3%BC%EC%A0%9C%5D_Selenium%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [과제] Selenium을 이용한 데이터 수집
- keonwoo Park 2020/09/15
- https://book.naver.com/ 네이버 책

In [44]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-broser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-broser/chromedriver')


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (125 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [51]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [209]:
from urllib.parse import quote
import time
import pandas as pd

def get_books(query, page_num =10):

  wd = webdriver.Chrome('chromedriver', options = chrome_options)
  wd.implicitly_wait(3)
  url_query = quote(query)
  url = 'https://book.naver.com/search/search.nhn?sm=sta_hty.book&sug=&where=nexearch&query=' + url_query
  wd.get(url)

  author = []
  title = []
  publisher = []
  idx = 0
  book_df = pd.DataFrame(columns=('Title', "Author","Publisher", "Date", "Price"))

  for page_no in range(1,page_num+1):
    if page_no % 10 != 1:
      page_ul = wd.find_element_by_class_name('paginate')
      page_a = page_ul.find_element_by_link_text(str(page_no))
      page_a.click()
      time.sleep(1)

    book_info = wd.find_elements_by_css_selector('#searchBiblioList > li > dl')
    for i in book_info:
      try:
        block = i.find_element_by_class_name('txt_block')
        date = block.text.split(' ')[-1]
        title_sub = i.find_element_by_tag_name('dt')
        title = title_sub.find_element_by_tag_name('a').text
        author = block.find_element_by_tag_name('a').text
        publisher = block.find_elements_by_tag_name('a')[-1].text
        price = i.find_element_by_class_name("txt_desc")
        lowest_price = price.find_element_by_class_name('price').text
      except:
        continue
      book_df.loc[idx]=[title,author, publisher, date, lowest_price]
      idx += 1
    try:        
      if page_no % 10 == 0:
        next_button = page_ul.find_element_by_class_name('next')
        next_button.click()
        time_sleep(1)
    except:
      break

  
  return book_df

In [211]:
query = input("검색 질의: ")
get_books(query, 12)

검색 질의: 추리


,Title,Author,Publisher,Date,Price
0,추리 천재 엉덩이 탐정 9,트롤,아이세움,2020.01.15,"10,800원(-10%)"
1,2020 하주응의 PSAT for NCS 추리.상황판단 핵심 기본서,하주응.위포트 연구소,위포트,2020.05.29,"21,420원(-10%)"
2,추리 천재 엉덩이 탐정 8,트롤,아이세움,2019.07.25,"10,800원(-10%)"
3,2020 하반기 최신판 위포트 대기업 인적성 & NCS 수리.추리 집중 완성,위포트 연구소,위포트,2020.07.16,"17,910원(-10%)"
4,추리 천재 엉덩이 탐정과 카레사건,트롤,아이세움,2019.05.01,"10,800원(-10%)"
...,...,...,...,...,...
159,너도밤 나무집 (Copper Beeches) - 셜록 홈즈 추리 단편소설 시리즈ㅣ영문판ㅣ,아서 코난 도일(Arthur Conan Doyle),부크크(Bookk),2019.05.31,"6,200원"
160,실종된 스리쿼터백 (Missing Three-Quarter) - 셜록 홈즈 추리 단...,아서 코난 도일 (Arthur Conan Doyle),부크크(Bookk),2019.05.31,"6,000원"
161,금테 코안경 (Golden Pince-Nez) - 셜록 홈즈 추리 단편소설 시리즈ㅣ...,아서 코난 도일 (Arthur Conan Doyle),부크크(Bookk),2019.05.31,"6,200원"
162,악마의 발 (Devil's Foot) - 셜록 홈즈 추리 단편소설 시리즈ㅣ영문판ㅣ,아서 코난 도일 (Arthur Conan Doyle),부크크(Bookk),2019.05.31,"6,200원"
